# Add the lasa.py file in same directory as the jupyter notebook

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from lasa import load_lasa

## Helper Functions for plotting and loading data

In [3]:
def plot_curves(x,show_start_end=True,**kwargs):
    '''
    plots 2d curves of trajectories

    params:
        x: array of shape (number of curves,n_steps_per_curve,2)
    '''
    if show_start_end:
        start_label,end_label = "start","end"
    else:
        start_label,end_label = None,None
    for t in range(x.shape[0]):
        plt.scatter(x[t][0,0],x[t][0,1],c='k',label=start_label)
        plt.scatter(x[t][-1,0],x[t][-1,1],c='b',label=end_label)
        plt.plot(x[t][:,0],x[t][:,1],**kwargs)
        if t==0:
            kwargs.pop("label",None)
            start_label,end_label = None,None

    plt.legend()


def streamplot(f,x_axis=(0,100),y_axis=(0,100),n=1000,width=1,**kwargs):
    '''
    helps visualizing the vector field.

    params:
        f: function to predict the velocities in DS(Dynamical system : x_dot = f(x),x of shape (n_points,2),x_dot of shape (n_points,2))
        x_axis: x axis limits
        y_axis: y axis limits
        n: number of points in each axis (so total n*n predictions happen)
        width: width of the vector
        **kwargs: goes into plt.streamplot
    '''
    a,b = np.linspace(x_axis[0],x_axis[1],n),np.linspace(y_axis[0],y_axis[1],n)
    X,Y = np.meshgrid(a,b)
    X_test = np.stack([X,Y],axis=-1).reshape(-1,2)
    Y_pred = f(X_test)
    U,V = np.split(Y_pred.reshape(n,n,2),2,axis=-1)
    U,V = U[...,0],V[...,0]
    speed = np.sqrt(U**2+V**2)
    lw = width*speed / speed.max()
    plt.streamplot(X,Y,U,V,linewidth=lw,**kwargs)


#loading the data and plotting
def load_data(letter):
    '''
    gets the trajectories coresponding to the given letter

    params:
      letter: character in ["c","j","s"]

    returns:
      data: array of shape (number of trajectories,number of timesteps,2)
      x: array of shape(number of trajectories*number of timesteps,2)
      xd: array of shape(number of trajectories*number of timesteps,2)

    '''
    letter2id = dict(c=2,j=6,s=24)
    assert letter.lower() in letter2id
    _,x,_,_,_,_ = load_lasa(letter2id[letter.lower()])
    xds = []
    for i in range(x.shape[0]):
        dt = 1/(x[i].shape[0]-1)
        xd = np.vstack((np.zeros((1, x[i].shape[1])), np.diff(x[i], axis=0) / dt))
        xds.append(xd)
    xd = np.stack(xds)
    plot_curves(x)
    data = x
    plt.show()
    return data,x,xd

In [ ]:
#letter should be one of c,j,s
data,x,xd = load_data("j")
data,x,xd = load_data("s")
data,x,xd = load_data("c")

In [ ]:
print(x.shape,xd.shape)

# ProMP (Probablistic Movement Primitives)
theory:[paper](https://papers.nips.cc/paper/2013/file/e53a0a2978c28872a4505bdb51db06dc-Paper.pdf)

In [6]:
class ProMP:
    def __init__(self,n_dims=2,nweights_per_dim=20):

        self.n_dims = n_dims
        self.nweights_per_dim = nweights_per_dim

    def get_features(self,T,overlap=0.7):
        '''
        gaussian feature vector for time ,and its derivative

        params:
            T:1d array , query time for features
            overlap: float(0.0-1.0) , heuristic for overlap between gaussians
        returns:
            rbfs.T : array(T x dim),feature vectors
            rbfs_derivative.T : array(T x dim), derivative of feature vectors
        '''
        assert T.ndim == 1
        assert np.max(T)<=1.0 and np.min(T)>=0.0
        h = -1.0 / (8.0 * self.nweights_per_dim** 2 * np.log(overlap))
        centers = np.linspace(0,1,self.nweights_per_dim)
        rbfs = np.exp(-(T[None,...] - centers[...,None]) ** 2 / (2.0 * h))
        rbfs_sum_per_step  = rbfs.sum(axis=0)
        rbfs_deriv = (centers[...,None] - T[None,...]) / h
        rbfs_deriv *= rbfs
        rbfs_deriv_sum_per_step = rbfs_deriv.sum(axis=0)
        rbfs_deriv = (
             rbfs_deriv * rbfs_sum_per_step
             - rbfs * rbfs_deriv_sum_per_step) / (rbfs_sum_per_step ** 2)
        rbfs /= rbfs_sum_per_step
        return rbfs.T,rbfs_deriv.T

    def fit(self,x,xd):
        '''
        fits w for each trajectory and estimates mean,covariance of w
        use the get_features method for feature calulation

        params:
            x:array of shape - (number of trajectories,n_steps,self.n_dims)
        '''
        #store the mean of w in self.mean_w -  array of shape(self.n_dim*self.nweights_per_dim,) (Note that this is a 1 dimensional array)
        #store the covaraince of w in self.cov_w - array of shape(self.n_dim*self.nweights_per_dim,self.n_dim*self.nweights_per_dim) (2d array)
        ################################
        #YOUR CODE HERE

        ################################


    def sample_trajectories(self,n_sample,mean_w=None,cov_w=None):
        '''
        samples trajectories given mean_w and cov_w , if not given uses the default (self.mean_w,self.cov_w)
        w ~ Normal(mean_w,cov_w)
        x_sample = mean of p( x / w )

        params:
            n_sample : int , number of trajectories to sample
            mean_w : array of shape - (self.nweights_per_dim*self.n_dims,)
            cov_w : array of shape - (self.nweights_per_dim*self.n_dims,self.nweights_per_dim*self.n_dims)
        returns:
            x_sample:array of shape - (n_sample,1000,self.n_dims)

        '''
        #use this T for getting features as we trained assuming this T
        T = np.linspace(0,1,1000)
        ################################
        #YOUR CODE HERE

        ################################


    def conditioning_on_xt(self,xt,t,cov = 0.0):
        '''
        changes the mean,covariance of the w based by conditioning on  x(t)

        params:
            xt : array of shape (self.n_dims,)
            t : 0.0 or 1.0 => 0.0 corresponds to starting point, 1.0 corresponds to the ending point

        returns:
            new_mean : array of shape (self.nweights_per_dim*self.n_dims,)
            new_cov : array of shape (self.nweights_per_dim*self.n_dims,self.nweights_per_dim*self.n_dims)
        '''

        assert xt.ndim==1
        assert t<=1.0 and t>=0

        y = np.zeros(2*len(xt))
        y[::2]=xt

        phi = np.vstack(self.get_features(np.array([t])))
        phi = self._nd_block_diagonal(phi,self.n_dims)
        phi = phi.T
        common_term = self.cov_w@phi@np.linalg.pinv(cov*np.eye(len(phi.T))+phi.T@self.cov_w@phi)
        new_mean_w = self.mean_w + common_term@(y-phi.T@self.mean_w)
        new_cov_w = self.cov_w - common_term@phi.T@self.cov_w
        return new_mean_w,new_cov_w




    def _nd_block_diagonal(self,partial_1d, n_dims):
        """Replicates matrix n_dims times to form a block-diagonal matrix.

        We also accept matrices of rectangular shape. In this case the result is
        not officially called a block-diagonal matrix anymore.

        Parameters
        ----------
        partial_1d : array, shape (n_block_rows, n_block_cols)
            Matrix that should be replicated.

        n_dims : int
            Number of times that the matrix has to be replicated.

        Returns
        -------
        full_nd : array, shape (n_block_rows * n_dims, n_block_cols * n_dims)
            Block-diagonal matrix with n_dims replications of the initial matrix.
        """
        assert partial_1d.ndim == 2
        n_block_rows, n_block_cols = partial_1d.shape

        full_nd = np.zeros((n_block_rows * n_dims, n_block_cols * n_dims))
        for j in range(n_dims):
            full_nd[n_block_rows * j:n_block_rows * (j + 1),
                    n_block_cols * j:n_block_cols * (j + 1)] = partial_1d
        return full_nd

In [ ]:
#fit the model
model = ProMP(n_dims=2,nweights_per_dim=20)
model.fit(x,xd)

In [ ]:
#sample different trajectories
x_lim = [np.min(x[:,:,0])-10,np.max(x[:,:,0])+10]
y_lim = [np.min(x[:,:,1])-10,np.max(x[:,:,1])+10]
plt.xlim(x_lim)
plt.ylim(y_lim)
x_sample = model.sample_trajectories(100)
plot_curves(x_sample,alpha = 0.2)

In [ ]:
#conditioning on starting point
x0 = x[0][-1]
new_mean,new_cov = model.conditioning_on_xt(x0,1.0,0.0)
x_sample = model.sample_trajectories(100,new_mean,new_cov)
plt.xlim(x_lim)
plt.ylim(y_lim)
plot_curves(x_sample,alpha=0.2)

# Fit the model to each of the data set and answer the following questions:

**For "C" Dataset:**

Q1.Does the system have a GAS (Global asymptotic Stable) Point ? Explain. \\
Ans:

Q2.Try conditioning on different initial points

Q3.Try changing the nweights_per_dim parameter and comment on the generalization of the algorithm. \\
Ans:

**For "J" Dataset:**

Q1.Does the system have a GAS (Global asymptotic Stable) Point ? Explain. \\
Ans:

Q2.Try conditioning on different initial points

Q3.Try changing the nweights_per_dim parameter and comment on the generalization of the algorithm. \\
Ans:

**For "S" Dataset:**

Q1.Does the system have a GAS (Global asymptotic Stable) Point ? Explain. \\
Ans:

Q2.Try conditioning on different initial points

Q3.Try changing the nweights_per_dim parameter and comment on the generalization of the algorithm. \\
Ans:

In [10]:
####################
#YOUR CODE HERE FOR ANSWERING THE ABOVE QUESTIONS
####################

# Submission details

please submit the copy of this file with the naming convention **SRNO(5digit)_Assignment7.ipynb** \\
example : 22592_Assignment8.ipynb